In [ ]:
import pandas as pd
import datetime as dt
import os
import logging
from pathlib import Path
import sys
sys.path.append('../src')
from utils.fetch_data import fetch_nwis_data
from utils.transform_data import transform_nwis_iv_data
from utils.write_to_datalake import write_to_datalake
import utils.duckdb_utils as db
# import utils.site_list as sl

In [ ]:
db.run_sql_file(Path('../db/views.sql'))

In [ ]:
# === CONFIGURATION ===
start_date = '2020-01-01'
end_date = (dt.date.today() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
service_code = 'iv'
notebook_dir = Path.cwd()
project_root = notebook_dir.resolve().parents[0]
datalake_path = project_root / 'data' / 'hydrology_datalake'
db_path = project_root / 'data' / 'hydrology.duckdb'
# output_root = Path(__file__).resolve().parents[2] / 'hydrology_datalake' # ok for main.py
# output_root = '/Volumes/T7_raw_I/waterdata_lake'
# datalake_path = output_root / 'timeseries_iv'
# ======================


# Configure logging ------------------------------------------------
os.makedirs('logs', exist_ok=True)
log_name = 'logs/' + dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.log'
logging.basicConfig(filename=log_name,
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
# -------------------------------------------------------------------

In [ ]:
# fetch site list associated with NWIS API
with db.connect_duckdb() as con:
    site_info = con.execute("SELECT site_id, site_cd,  FROM site WHERE lower(source) LIKE '%nwis%'").fetchall()
    sids = [site[0] for site in site_info]
    scodes = [site[1] for site in site_info]
    print(scodes, sids)

In [ ]:
#scodes = ['09315000']

In [ ]:
for site in scodes:
    logging.info(f"Processing site: {site}")
    
    query = (
    "SELECT p.parameter_cd"
    " FROM parameter p"
    " INNER JOIN site_parameter sp ON p.parameter_id = sp.parameter_id"
    " INNER JOIN site s ON sp.site_id = s.site_id"
    f" WHERE s.site_cd = '{site}'"
    )

    with db.connect_duckdb() as con:
        params = con.execute(query).fetchall()
        parameter_codes = [param[0] for param in params]
        print(parameter_codes)

    all_data = []
# Loop through each site and its associated parameter codes
    for pcode in parameter_codes:
        df_raw = fetch_nwis_data(
            site=site,
            pcode=pcode,
            start_date=start_date,
            end_date=end_date,
        )
        if df_raw is None:
            continue

        df_transformed = transform_nwis_iv_data(df_raw, site, pcode)
        if not df_transformed.empty:
            all_data.append(df_transformed)

    if all_data:
        df_combined = pd.concat(all_data, ignore_index=True)
        write_to_datalake(df_combined, site)

    
    

In [ ]:
df_raw.columns

In [ ]:
with db.connect_duckdb() as con:
    result = con.execute("SELECT * FROM vw_nwis_annual_stats_local WHERE approval_status = 'A' AND year = 2023 AND parameter_nm = 'Water temperature'").df()
    param_codes = con.execute("SELECT DISTINCT parameter_cd FROM vw_nwis_iv_local").fetchall()
    param_codes = [param[0] for param in param_codes]
    print(param_codes)

In [ ]:
query = """
    SELECT site_nm, parameter_nm, max(max_value) AS max_value
    FROM vw_nwis_annual_stats_local
    WHERE approval_status = 'A' AND hydro_area_nm = 'Colorado River'
    GROUP BY site_nm, year, parameter_nm
    """

with db.connect_duckdb() as con:
    result = con.execute(query).df()
    print(result.head())

In [ ]:
with db.connect_duckdb() as con:
    result = con.execute("SELECT * FROM vw_nwis_annual_stats_local WHERE min_date IS NULL").df()
    print(result.head())

In [ ]:
result.columns
print(result['parameter_cd'].unique())

In [ ]:
site = '09095500'
query = (
    "SELECT p.parameter_cd"
    " FROM parameter p"
    " INNER JOIN site_parameter sp ON p.parameter_id = sp.parameter_id"
    " INNER JOIN site s ON sp.site_id = s.site_id"
    f" WHERE s.site_cd = '{site}'"
)

with db.connect_duckdb() as con:
        params = con.execute(query).fetchall()
        pcodes = [param[0] for param in params]
        print(pcodes)





In [ ]:
with db.connect_duckdb() as con:
    tables = con.execute("SHOW TABLES").fetchall()
    table_names = [table[0]for table in tables]

    print(table_names)

with db.connect_duckdb() as con:
    columns = con.execute("DESCRIBE site").fetchall()
    column_names = [col[0] for col in columns]

    print(column_names)

In [ ]:
ck = pd.read_csv("../artifacts/site_parameter.csv",
                 dtype={"parameter_cd": "string",
                        "site_cd": "string"})
print(ck.head())

In [ ]:
db.refresh_db_from_csv(table_name="site_parameter", csv_path="../artifacts")

In [ ]:
with db.connect_duckdb() as con:
    # Check if the table is refreshed
    refreshed_data = con.execute("SELECT * FROM site").df()
    print(f"Refreshed data in 'parameter' table: {refreshed_data[:5]}")  # Print first 5 rows for verification

In [ ]:
data = [
    {
        'site_id': 46,
        'site_cd': '2003',
        'site_nm': 'SHADOW MOUNTAIN RESERVOIR',
        'site_dsc': None,
        'agency_cd': 'BOR',
        'agency_nm': 'US Bureau of Reclamation',
        'lat_dd': 40.22698,
        'lon_dd': -105.84385,
        'elev_m': None,
        'site_type': 'lake',
        'hydro_area_cd': 'SMR',
        'hydro_area_nm': 'Shadow Mountain Reservoir',
        'source': 'https://www.usbr.gov/uc/water/hydrodata/reservoir_data/<site_cd>/csv/<parameter_cd>.csv'
    },
    {
        'site_id': 47,
        'site_cd': '100118',
        'site_nm': 'WILLOW CREEK RESERVOIR',
        'site_dsc': None,
        'agency_cd': 'BOR',
        'agency_nm': 'US Bureau of Reclamation',
        'lat_dd': 40.146932,
        'lon_dd': -105.942513,
        'elev_m': None,
        'site_type': 'lake',
        'hydro_area_cd': 'WCR',
        'hydro_area_nm': 'Willow Creek Reservoir',
        'source': 'https://www.usbr.gov/uc/water/hydrodata/reservoir_data/<site_cd>/csv/<parameter_cd>.csv'
    }
]

df = pd.DataFrame(data)
df['created_ts'] = pd.Timestamp.now()
df['updated_ts'] = pd.Timestamp.now()

In [ ]:
with connect_duckdb() as con:
    #con.register('site_tmp', df)
    result = con.execute("SELECT * FROM site").df()

In [ ]:
data = [
    {
        'parameter_id': 30,
        'parameter_cd': '63160',
        'parameter_nm': 'Stream surface elevation',
        'parameter_dsc': 'NAVD 1988',
        'unit_cd': 'ft',
        'unit_nm': 'Feet'
    },
    {
        'parameter_id': 31,
        'parameter_cd': '00054',
        'parameter_nm': 'Reservoir storage',
        'parameter_dsc': None,
        'unit_cd': 'af',
        'unit_nm': 'Acre-feet'
    }
]
df = pd.DataFrame(data)
df['created_ts'] = pd.Timestamp.now()
df['updated_ts'] = pd.Timestamp.now()

In [ ]:
with connect_duckdb() as con:
    df = con.execute("SELECT * FROM parameter").df()

In [ ]:
query = (
    "SELECT sp.site_id, s.site_nm, p.parameter_id, p.parameter_nm, p.parameter_cd"
    " FROM site_parameter sp"
    " LEFT JOIN parameter p"
    " ON sp.parameter_id = p.parameter_id"
    " LEFT JOIN site s"
    " ON sp.site_id = s.site_id"
    " ORDER BY sp.site_id, p.parameter_id"
)
#query = "select * from parameter"
with connect_duckdb() as con:
    result = con.execute(query).df()
    
"""

print(query)
with connect_duckdb() as con:
    df = con.execute(query).df()
    print(df.head())
"""


In [ ]:
result['created_ts'] = pd.Timestamp.now()
result['updated_ts'] = pd.Timestamp.now()
result['site_parameter_id'] = range(1, len(result) + 1)
result = result[['site_parameter_id', 'site_id', 'parameter_id', 'created_ts', 'updated_ts']].copy()


In [ ]:

with connect_duckdb() as con:
    con.register('site_parameter_tmp', result)
    con.execute("INSERT INTO site_parameter SELECT * FROM site_parameter_tmp")
    print(con.execute("SELECT * FROM site_parameter").df().head())

In [ ]:
result['created_ts'] = pd.Timestamp.now()
result['updated_ts'] = pd.Timestamp.now()
result['site_parameter_id'] = range(1, len(result) + 1)
result = result[['site_parameter_id', 'site_id', 'parameter_id', 'created_ts', 'updated_ts']].copy()


In [ ]:

with connect_duckdb() as con:
    con.register('site_parameter_tmp', result)
    con.execute("INSERT INTO site_parameter SELECT * FROM site_parameter_tmp")
    print(con.execute("SELECT * FROM site_parameter").df().head())

In [ ]:
query = """
    SELECT hydro_area_nm, site_nm, year, max_value, max_date
    FROM vw_nwis_annual_stats_local
    WHERE parameter_cd = '00060'
    order by year DESC, hydro_area_nm, site_nm
    """
query = "SELECT * FROM parameter"

with connect_duckdb() as con:
    result = con.execute(query).df()
    print(result.head())

In [ ]:
nwis_params = pd.read_csv("../artifacts/USGS_parameter_codes.tsv", sep="\t", dtype=str)
nwis_params = nwis_params.loc[~nwis_params['parameter_cd'].isin(['00060', '00010'])].reset_index(drop=True)
nwis_params['parameter_id'] = range(23, len(nwis_params) + 23)
nwis_params['create_ts'] = pd.Timestamp.now()
nwis_params['update_ts'] = pd.Timestamp.now()

In [ ]:
column_list = result.columns.tolist()
nwis_params = nwis_params[column_list].copy()

In [ ]:
with connect_duckdb() as con:
    con.register('nwis_params', nwis_params)
    con.execute("INSERT INTO parameter SELECT * FROM nwis_params")

In [ ]:
query = "SELECT * FROM site"
query = "SELECT * FROM parameter"

In [ ]:
run_sql_file(Path('../db/views.sql'))

In [ ]:
with connect_duckdb() as con:
    result = con.execute(query).df()
print(result.head())

In [ ]:
param = pd.read_csv('../artifacts/parameters.csv')

with connect_duckdb() as con:
    con.register('param', param)
    con.execute('INSERT INTO parameter SELECT * FROM param')